<a href="https://colab.research.google.com/github/szyluc/ISU_Student-Scheduler/blob/main/IT_244_Scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade --quiet google-auth google-auth-oauthlib google-auth-httplib2


In [3]:
# PIP INSTALLS HERE
%%capture
!pip install google-api-python-client
!pip install gradio # FOR USER INTERFACE
!pip install transformers
!pip install torch

In [4]:
# IMPORTS HERE
from google.oauth2.credentials import Credentials # 3 IMPORTS FOR GOOGLE CALENDAR
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from __future__ import print_function
import datetime
import os.path
import re
import torch
import gradio as gr

In [5]:
# MODEL LOADING HERE
from transformers import AutoTokenizer, AutoModelForCausalLM

# I AM USING A QWEN MODEL & COMPARING ITS RESULTS WITH A DISTILLED DEEPSEEK R1 MODEL
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct-1M", torch_dtype = torch.float16, device_map = "auto") #low_cpu_mem_usage = True
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct-1M")
#tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
#model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", torch_dtype = torch.float16, device_map = "auto")

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

creds_path = '/content/drive/MyDrive/credentials.json'
token_path = '/content/drive/MyDrive/token.json'

Mounted at /content/drive


In [7]:
from google.colab import auth
auth.authenticate_user()  # This should trigger the OAuth flow and prompt for authentication.


In [8]:
from google.colab import auth
import google.auth
SCOPES = ['https://www.googleapis.com/auth/calendar.events']

creds, _ = google.auth.default(scopes=SCOPES)

def auth_google_calendar():
    # Step 1: Authenticate the user using Colab's built-in auth
    auth.authenticate_user()

    # Step 2: Get credentials from google.auth and apply the correct scope
    creds, _ = google.auth.default(scopes=SCOPES)

    # Step 3: Check if the credentials are expired and refresh them if necessary
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())

    # Step 4: Build the Calendar service using the credentials
    return build('calendar', 'v3', credentials=creds)

print(creds.scopes)



['https://www.googleapis.com/auth/calendar.events']


In [10]:
#PROMPT I AM USING FOR TESTING
#I have a hectic week coming up... On Monday and Wednesday, I have Math from 10AM to 12PM. History from 1PM to 2PM. Science from 2:30PM to 3:45PM, and finally work from 5PM to 9PM. On Tuesday, I have to study for a history exam later in the week on Thursday. Finally, on Friday, I have to make time for intramural soccer practice at 2PM to 4PM.

# INTRODUCTORY MESSAGE HERE
user_intro = "Hello, I am a task scheduler here to assist you on your academic journey. Simply tell me what responsibilities you have this week for any given day, and I will generate a schedule fit to your needs."
print(user_intro)

# PROMPTING USER FOR THEIR SCHEDULE INPUT
user_input = gr.Textbox(label = "Enter your responsibilities for each day...", placeholder = "Monday: Biology from 11AM to 12:30PM, Math from 1PM to 2:15PM, Work from 4PM to 9PM, Need to study for math exam..., etc.")

# GENERATING THE USER SCHEDULE, FORGING THE PROMPT FOR THE MODEL TO FOLLOW (CAN ALTER TO CREATE MORE PERSONALIZED RESULT IF NEED BE)
def generate_user_schedule(user_input):
  prompt_for_model = f"""Generate a daily schedule for each day of the week based on the following user responsibilities:

{user_input}

Format the output in the following structure:
- Start each day with the day name on its own line, like: **Monday**
- Use this format for each task on that day:
  - HH:MM AM/PM - HH:MM AM/PM: Task Description

Make sure the time ranges are written with both start and end times, using AM/PM and the colon (:) format (e.g., 10:00 AM - 12:00 PM).
"""

  input = tokenizer(prompt_for_model, return_tensors = "pt").to("cuda") # TOKENIZES THE USER INPUT
  output = model.generate(**input, max_new_tokens = 1024) # MODEL GENERATES TOKENIZED OUTPUT BASED ON INPUT
  return tokenizer.decode(output[0], skip_special_tokens = True) # DECODES THE TOKENIZED OUTPUT AND RETURNS IT FOR USER

# EXPORTING SCHEDULE TO GOOGLE CALENDAR
def export_to_google_calendar(schedule_text):
    auth = auth_google_calendar()

    today = datetime.date.today()
    monday = today + datetime.timedelta(days=(7 - today.weekday()) % 7)

    day_offset = {
        "monday": 0, "tuesday": 1, "wednesday": 2,
        "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6
    }

    current_day = None
    added_events = 16

    for line in schedule_text.splitlines():
        line = line.strip()

        # Detect day header (e.g. "Wednesday")
        day_match = re.match(r'^\*{0,2}(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\*{0,2}$', line, re.IGNORECASE)
        if day_match:
            current_day = day_match.group(1).lower()
            continue

        # Match event lines like: "- 10:00 AM - 12:00 PM: Math"
        event_match = re.match(
            r'^-?\s*(\d{1,2}:\d{2}\s*[APMapm]{2})\s*-\s*(\d{1,2}:\d{2}\s*[APMapm]{2})\s*:\s*(.+)$',
            line
        )

        if current_day and event_match:
            try:
                # Extract and clean up time fields (handling extra spaces)
                start_time_str = event_match.group(1).strip()
                end_time_str = event_match.group(2).strip()
                task = event_match.group(3).strip()

                # Parse datetime (removing extra spaces just in case)
                event_date = monday + datetime.timedelta(days=day_offset[current_day])

                # Try parsing times directly (handles AM/PM format)
                start_dt = datetime.datetime.strptime(start_time_str, "%I:%M %p")
                end_dt = datetime.datetime.strptime(end_time_str, "%I:%M %p")

                # Create start and end datetime with proper date attached
                start_datetime = datetime.datetime.combine(event_date, start_dt.time())
                end_datetime = datetime.datetime.combine(event_date, end_dt.time())

                # Event to Google Calendar
                event = {
                    'summary': task,
                    'start': {'dateTime': start_datetime.isoformat(), 'timeZone': 'America/Chicago'},
                    'end': {'dateTime': end_datetime.isoformat(), 'timeZone': 'America/Chicago'}
                }

                auth.events().insert(calendarId='primary', body=event).execute()
                added_events += 1
            except Exception as e:
                print(f"Error adding event: {e}")

    return f"{added_events} event(s) successfully added to your Google Calendar!"





# HELPER FUNCTION USED TO PARSE TIME IN EXPORT FUNCTION
def parse_time(t):
  try:
    return datetime.datetime.strptime(t.strip(), "%I%p")
  except ValueError:
    return datetime.datetime.strptime(t.strip(), "%I:%M%p")


# GRADIO USER INTERFACE
with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      user_input = gr.Textbox(
          label = "Enter your responsibilities for each day...",
          placeholder = "Monday: Biology from 11AM to 12:30PM, Math from 1PM to 2:15PM, Work from 4PM to 9PM, Need to study for math exam..., etc.",
          lines = 10
      )
      generate_button = gr.Button("Generate Schedule")

    with gr.Column():
      generated_schedule_output = gr.Textbox(label = "Your Generated Schedule", lines = 10)
      with gr.Row():
        export_to_google_calendar_button = gr.Button("Export to your Google Calendar!")

    generate_button.click(fn = generate_user_schedule, inputs = user_input, outputs = generated_schedule_output) # GENERATE BUTTON OUTPUTS USER'S PERSONALIZED SCHEDULE
    export_to_google_calendar_button.click(fn = export_to_google_calendar, inputs = generated_schedule_output, outputs = gr.Textbox(label = "Export Status")) # NO OUTPUT, SIMPLY EXPORTING TO GOOGLE CALENDAR


demo.launch()

Hello, I am a task scheduler here to assist you on your academic journey. Simply tell me what responsibilities you have this week for any given day, and I will generate a schedule fit to your needs.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a44d3ca547d295409e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
